In [16]:
import os
from google.cloud import pubsub_v1

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../erudite-flag-364413-428e8c5a8dbb.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "erudite-flag-364413"

publisher = pubsub_v1.PublisherClient()
topic_name = 'projects/{project_id}/topics/{topic}'.format(
    project_id=os.getenv('GOOGLE_CLOUD_PROJECT'),
    topic='RC.MODEL',  # Set this to something appropriate.
)

publisher.create_topic(name=topic_name)
future = publisher.publish(topic_name, b'My first message!', spam='eggs')
future.result()

'7630464457962685'

In [ ]:
import jax.numpy as jnp
import json

data = {
    'a': {'rate': , 'concentration': } 
    'b': {'rate': , 'concentration': } 
}

user_dict = {'name': 'dinesh', 'code': 'dr-01'}
user_encode_data = json.dumps(user_dict, indent=2).encode('utf-8')
print(user_encode_data)